In [1]:
from feedback import *
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
save_weight_path ='./models/weights_Multiclass_Covid19(Non-kmer3)_IndexRemark.2022.03.24[NATCG]/'
# save_weight_path = './models/weights_Multiclass_Covid19(Non-kmer3)_IndexRemark.2022.03.24[NACGTRYKMSWBDHV]/'

weights_name = "weights_Multiclass_Covid19(Non-kmer3)[NACGT].2022.03.24.pt"
# weights_name = "weights_Multiclass_Covid19(Non-kmer3)[NACGTRYKMSWBDHV].2022.03.24.pt"

path2weights = os.path.join(save_weight_path,weights_name)

In [3]:
# npy_path = './np_image_totalunit/multiclass_totalunit/'
npy_path = './np_image_totalunit/multiclass_nactg/'

In [4]:
npy_data_list = [os.path.join(npy_path,'image_npy',i ) for i in sorted(os.listdir(os.path.join(npy_path,'image_npy')))]
label_ = np.load(os.path.join(npy_path,'label.npy'))

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.25, random_state=42)

train_indx, test_indx = next(sss.split(npy_data_list, label_))
train_ids = [npy_data_list[ind] for ind in train_indx]
train_labels = [label_[ind] for ind in train_indx]
print(len(train_ids), len(train_labels)) 

test_ids = [npy_data_list[ind] for ind in test_indx]
test_labels = [label_[ind] for ind in test_indx]
print(len(test_ids), len(test_labels))

1053 1053
351 351


In [6]:
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as transforms
import glob
from PIL import Image
import torch
import numpy as np
import random
np.random.seed(2020)
random.seed(2020)
torch.manual_seed(2020)

class TransferDataset(Dataset):
    def __init__(self, ids, labels, transform):
        self.transform = transform
        self.ids = ids
        self.labels = labels
    def __len__(self):
        return len(self.ids)
    def __getitem__(self, idx):
        # print(self.ids[idx])
        singel_image_ = np.load(self.ids[idx]).astype(np.float32)
        seed = np.random.randint(1e9)       
        random.seed(seed)
        np.random.seed(seed)
        singel_image_ = self.transform(singel_image_)
        # singel_image_ = torch.unsqueeze(self.transform(singel_image_)[0], axis=0)
        label = int(self.labels[idx])
        # print(label)

        return singel_image_, label

In [7]:
transformer = transforms.Compose([
            transforms.ToTensor(),
            # transforms.Normalize(mean, std),
            ])     

train_ds = TransferDataset(ids= train_ids, labels= train_labels, transform= transformer)
test_ds = TransferDataset(ids= test_ids, labels= test_labels, transform= transformer)
print(len(train_ds), len(test_ds))

1053 351


In [8]:
imgs, label = train_ds[10]
batch_size = 32
train_dl = DataLoader(train_ds, batch_size= batch_size, 
                        shuffle=True)
test_dl = DataLoader(test_ds, batch_size= 2*batch_size, 
                        shuffle=False)  

In [9]:
from torchvision.models import alexnet
from torch.nn import Module
from torch import nn


In [10]:
# eval
models = alexnet(pretrained=False, num_classes=max(label_)+1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load(path2weights, map_location=torch.device('cpu'))
# checkpoint = torch.load('./models/weights_Multiclass_Covid19(Non-kmer3)_IndexRemark.2022.03.24[NACGTRYKMSWBDHV]/weights_Multiclass_Covid19(Non-kmer3)[NACGTRYKMSWBDHV].2022.03.24.pt', map_location=torch.device('cpu'))
models.load_state_dict(checkpoint['model_state_dict'])
models.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [11]:
test_dl = DataLoader(test_ds, batch_size= 351, 
                        shuffle=False)  
models.eval()
with torch.no_grad():
    for xb, yb in tqdm(test_dl):
        xb=xb.to(device)
        yb=yb.to(device)
    predict_result = np.argmax(models(xb), axis=1).numpy()
    ground_truth = yb.numpy()

  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(ground_truth, predict_result)

0.9715099715099715

In [13]:
len(test_ds)

351

In [14]:
for xb,yb in train_dl:
    # print(xb.shape, yb.shape)
    # print(xb[0][0][0][0])
    print(yb[0])
    break

for xb,yb in test_dl:
    print(xb.shape, yb.shape)
    break

tensor(3)
torch.Size([351, 3, 100, 100]) torch.Size([351])


In [15]:
train_index = np.random.choice(len(train_ds), 100, replace=False)

In [16]:
test_index = np.random.choice(len(test_ds), 5, replace=False)

In [17]:
test_data = [np.load(test_ids[i]).astype(np.float32) for i in test_index]

In [18]:
for image, label in train_dl:
    print(image.shape)
    break

torch.Size([32, 3, 100, 100])


In [19]:
for image2, label2 in test_dl:
    print(image2[0:5].shape)
    break

torch.Size([5, 3, 100, 100])


In [20]:
import shap
import numpy as np



In [ ]:
e = shap.DeepExplainer(models, image)
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)
shap_values = e.shap_values(image2[0:5])

# plot the feature attributions
shap.image_plot(shap_values, -image2[0:5])